# Table S10. Prioritization of actionable susceptibility-associated

regions detected by GWAS

In [ ]:

library(tidyverse)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Attaching package: 'flextable'

The following object is masked from 'package:purrr':

    compose


Attaching package: 'data.table'

The following objects are masked from 'package:lubridate':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

The following objects are masked from 'package:dplyr':

    between, first, last

The following object is masked from 'package:purrr':

    transpose


Attaching package: 'kableExtra'

The following objects are masked from 'package:flextable':

    as_image, footnote

The following object is masked from 'package:dplyr':

    group_rows

here() starts at /Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript

$strain_table
$strain_table$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.html"

$strain_table$tsv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.tsv.zip"

$strain_table$docx
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.docx"

$strain_table$csv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.csv"


$tox_table_ft
$tox_table_ft$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.html"

$tox_table_ft$tsv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.tsv.zip"

$tox_table_ft$docx
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.docx"

$tox_table_ft$csv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.csv"


$trait_table
$trait_table$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S3/table_S3.html"

$trait_table$tsv
[1] "

# Inputs

In [ ]:

# Load the ranked QTL data
inbred_ranked_fn <- "data/processed/qtl_ranking/inbred/ranked_qtl.tsv"

# Load the tox condition metadata
tox_metadata_fn <- "data/processed/tox_data/con_metadata.csv"


# Main

In [ ]:

# Load the ranked intervals
inbred_ranked <- data.table::fread(inbred_ranked_fn)

# Load the tox condition metadata
tox_metadata <- data.table::fread(tox_metadata_fn) %>%
  dplyr::filter(type == "Toxicant") %>%
  dplyr::select(trait, nice_drug_label2, big_class, class, moa_class) %>%
  # adjust the uM to µM
  dplyr::mutate(nice_drug_label2 = dplyr::case_when(
    nice_drug_label2 == "uM" ~ "µM",
    TRUE ~ nice_drug_label2
  ))

# Join tox metadata with QTL data
inbred_ranked <- inbred_ranked %>%
  dplyr::left_join(tox_metadata, by = c("trait" = "trait"))

# Calculate proportions and format data
inbred_ranked <- inbred_ranked %>%
  dplyr::mutate(
    condition_marker = paste0(nice_drug_label2, " (", marker, ")"),
    prop_strains_ref = n_strains_ref / total_strains,
    prop_strains_alt = n_strains_alt / total_strains,
    # Ensure numeric columns are properly typed and rounded
    log10p = round(as.numeric(log10p), 3),
    strain_freq = paste0(round(prop_strains_ref, 3), "/", round(prop_strains_alt, 3)),
    prop_alt_overlap = round(as.numeric(prop_alt_overlap), 3)
  )


# Create Flextable

In [ ]:

# Create flextable
inbred_qtl_ranking_ft <- flextable(
  inbred_ranked,
  col_keys = c(
    "rank",
    "condition_marker",
    "log10p",
    "Fine Mapping LD Structure",
    "strain_freq",
    "prop_alt_overlap"
  )
) %>%
  flextable::set_header_labels(
    values = list(
      "rank" = "Rank",
      "condition_marker" = "Condition (Peak Marker)",
      "log10p" = "-log10(p-value)",
      "Fine Mapping LD Structure" = "Fine Mapping LD Structure",
      "strain_freq" = "REF/ALT freq.",
      "prop_alt_overlap" = "Prop ALT strains HVR"
    )
  ) %>%
  # Make header bold
  flextable::bold(part = "header") %>%
  flextable::line_spacing(space = 1, part = "all") %>%
  flextable::set_table_properties(
    layout = "autofit",
    width = 1,
    opts_word = list(
      split = FALSE,
      keep_with_next = TRUE
    )
  ) %>%
  # Add vertical line after the first column
  flextable::vline(j = 1, part = "body")

inbred_qtl_ranking_ft


Rank,Condition (Peak Marker),-log10(p-value),Fine Mapping LD Structure,REF/ALT freq.,Prop ALT strains HVR
1,Chlorothalonil (II:5392976),7.94,<250kb,0.216/0.784,0.98
2,Mercury (III:2461452),6.20,<250kb,0.865/0.135,0.77
3,Chlorothalonil (II:13921653),6.13,<250kb,0.237/0.763,0.82
4,Carboxin (III:2527992),5.91,<250kb,0.451/0.549,1.00
5,Arsenic (III:1642838),5.87,>500kb,0.134/0.866,1.00
6,Mercury (II:626831),5.42,<250kb,0.886/0.114,1.00
7,Arsenic (II:13692527),5.34,<250kb,0.129/0.871,0.81
8,Propoxur (V:5315004),5.22,<250kb,0.876/0.124,0.58
9,Chlorothalonil (III:4478684),5.21,250..500kb,0.335/0.665,0.35
10,Mercury (II:3977000),5.18,<250kb,0.886/0.114,1.00


# Save Tables

In [ ]:
# Prepare CSV output with nice column names
inbred_ranked_csv <- inbred_ranked %>%
  dplyr::select(
    rank,
    condition_marker,
    log10p,
    `Fine Mapping LD Structure`,
    strain_freq,
    prop_alt_overlap
  ) %>%
  dplyr::rename(
    "Rank" = rank,
    "Condition (Peak Marker)" = condition_marker,
    "-log10(p-value)" = log10p,
    "REF/ALT freq." = strain_freq,
    "Prop ALT strains HVR" = prop_alt_overlap
  )

# Save the flextable and CSV using standardized naming
save_supp_table_flextable(inbred_qtl_ranking_ft, "inbred_qtl_ranking_ft")
save_supp_table_csv(inbred_ranked_csv, "inbred_qtl_ranking_ft")
